# **_Event Extraction_**

`Import packeges`  
Using spacy dependency parser model

In [51]:
import spacy
from spacy import displacy  # Use displacy to visualize the dependency tree
import re
import warnings

nlp = spacy.load('en_core_web_trf')  # load language model
warnings.filterwarnings("ignore")
# ignore warning, mainly ignore the warning about GPU

`Load data`  
We load the corpus to a list

In [52]:
corpus = []
with open("./data/dataset.txt", mode='r') as file:
    sentence = file.readline()
    while sentence:
        corpus += [sentence]
        sentence = file.readline()


`Preprocess corpus`  
We mainly use regular expression to process the corpus

In [53]:
def get_trigger(sentence: str):
    trigger = []
    tags = [' <e> ', ' </e> ']
    for tag in tags:
        lst = sentence.split(tag)
        for i in range(0, len(lst) - 1):
            trigger.append(lst[i].split(' ')[-1])
    return trigger


trigger = [get_trigger(sentence) for sentence in corpus]
corpus = [re.sub(" <[\S]*> ", " ", s) for s in corpus]
corpus = [re.sub(" \. ", ".", s) for s in corpus]
corpus = [re.sub("[\.]{3}", " ... ", s) for s in corpus]
corpus = [re.sub(" , ", ", ", s) for s in corpus]
corpus = [re.sub(" - ", " ", s) for s in corpus]
corpus = [re.sub("[ ]{2,}", " ", s) for s in corpus]
corpus = [re.sub("\n", "", s) for s in corpus]

`Get dependency tree`  

In [55]:
"""
The speed of dependency parsing is about 10 sentence per second.
We can process about 1000 sentences one time and finish all test work 
to check the method we apply is really work. 
"""
dependency = [nlp(sentence) for sentence in corpus[:10]]


def transform(sentence):
    def to_nltk_tree(node):
        if node.n_lefts + node.n_rights > 0:
            return (node.orth_, [(child.tag_, to_nltk_tree(child))
                                 for child in node.children])
        else:
            return node.orth_

    return [to_nltk_tree(sent.root) for sent in sentence.sents]


trees = [transform(sentence) for sentence in dependency]
